# 参数管理
我们首先关注具有单隐藏层的多层感知机

In [2]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X = torch.rand(size=(2,4))
net(X)


tensor([[-0.0807],
        [-0.0434]], grad_fn=<AddmmBackward0>)

## 1 参数访问

In [3]:
# 把sequential看成一个list ----- 在定义 的成员变量 _module :OrderedDict
net[2].state_dict()#nn.Linear(8,1)

net[0].state_dict()#nn.Linear(4,8)

OrderedDict([('weight',
              tensor([[ 0.2376, -0.2647,  0.1141,  0.2113],
                      [-0.0313,  0.3448,  0.1423, -0.0818],
                      [-0.2364,  0.3065,  0.3730, -0.1393],
                      [-0.3964,  0.3756, -0.3444,  0.1501],
                      [ 0.1909,  0.4230, -0.4910,  0.3870],
                      [-0.1400,  0.4585, -0.1381, -0.0952],
                      [-0.4512, -0.2624,  0.0740,  0.0791],
                      [-0.1508,  0.4098,  0.1343,  0.1750]])),
             ('bias',
              tensor([ 0.3552, -0.0806,  0.2991, -0.0879,  0.2326, -0.2406,  0.0345,  0.3418]))])

## 2 目标参数

In [4]:
type(net[2].bias)
net[2].bias # 一个parameter变量
net[2].bias.data # 访问变量的数值


tensor([0.1346])

In [5]:
net[2].weight.grad == None# 访问梯度


True

## 3 一次性访问所有参数

In [6]:
#通过name_parameters()访问参数
#访问第一层的参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
#访问所有层的参数，0代表第一层，第二层是relu，没有参数
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [7]:
#通过名字来访问参数
net.state_dict()['2.bias'].data

tensor([0.1346])

## 4 从嵌套块收集参数

In [8]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4),
                         nn.ReLU())

def block2():
    net = nn.Sequential()
    #可以传名字
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)
rgnet

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)

## 5 内置初始化

In [9]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0072,  0.0080,  0.0172, -0.0019]), tensor(0.))

In [13]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

## 6 对某些块应用不同的初始化方法

In [10]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight,42)

net[0].apply(xavier)
net[2].apply(init_42)
net[0].weight.data[0],net[2].weight.data

(tensor([ 0.2990, -0.3312, -0.1891,  0.1374]),
 tensor([[42., 42., 42., 42., 42., 42., 42., 42.]]))

自定义初始化

In [11]:
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init",
            *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

#把net所有层的参数都遍历一遍，做初始化
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 7.9440, -0.0000,  9.5454, -0.0000],
        [ 6.4220, -8.5513, -0.0000,  0.0000]], grad_fn=<SliceBackward0>)

## 6 参数绑定

In [12]:
# CNN实现 参数共享
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared,
                    nn.ReLU(), nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
